In [1]:
from nansat import Nansat, Domain
import matplotlib.pyplot as plt
import numpy as np
from boreali import  Boreali, lm
import custrepr

/home/artemm/miniconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
# different kinds of test data was processed by boreali algorithms 
# october data
n_osw_oct = Nansat('/home/artemm/Documents/michigan/tests/A2014301181500.L2_LAC_OC.x.nccpa_OSW.nc')
n_deep_oct = Nansat('/home/artemm/Documents/michigan/tests/A2014301181500.L2_LAC_OC.x.nccpa_deep.nc')
# may data
n_osw_may = Nansat('/home/artemm/Documents/michigan/tests/A2014121184000.L2_LAC_OC.x.nccpa_OSW.nc')
n_deep_may = Nansat('/home/artemm/Documents/michigan/tests/A2014121184000.L2_LAC_OC.x.nccpa_deep.nc')

# base files
n_rrs_oct = Nansat('/home/artemm/Documents/michigan/data/2014/oct/A2014301181500.L2_LAC_OC.x.nc')
n_rrs_may = Nansat('/home/artemm/Documents/michigan/data/2014/may/A2014121184000.L2_LAC_OC.x.nc')

# bathymetry
bottom = Nansat('/home/artemm/Documents/michigan/tests/michigan_lld.grd')

dom = Domain('+proj=latlong +datum=WGS84 +ellps=WGS84 +no_defs', '-lle -86.3 44.6 -85.2 45.3 -ts 300 200')

n_rrs_oct.reproject(dom)
n_rrs_may.reproject(dom)
bottom.reproject(dom)

=>michigan.grd<=


In [17]:
# droving of bottom
h = np.copy(bottom[1])
h[np.where(h > np.float32(0.0))] = np.nan
h[np.where(h < np.float32(0.0))] *= np.float32(-1)
plt.figure('bottom')
plt.imshow(h)
plt.colorbar()
plt.show()

/home/artemm/miniconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: RuntimeWarning: invalid value encountered in less


In [18]:
# droving of Rrsw spectrum for the deep and shallow region 
num_bands = range(2, 12)
test_deep = [n_rrs_oct[band][43][5] for band in range(2, 12)]
h_deep = bottom[1][43][5]
h_shallow = bottom[1][111][41]
test_shallow = [n_rrs_oct[band][111][41] for band in range(2, 12)]
wavelens = [412, 443, 469, 488, 531, 547, 555, 645, 667, 678]
custom_n = Nansat(domain=n_rrs_oct)

# Rrsw processing
band_rrs_numbers = list(map(lambda x: n_rrs_oct._get_band_number('Rrs_' + str(x)), wavelens))

for index in range(0, len(wavelens)):
    rrsw = n_rrs_oct[band_rrs_numbers[index]] / (0.52 + 1.7 * n_rrs_oct[band_rrs_numbers[index]])
    custom_n.add_band(rrsw, parameters={'name': 'Rrsw_' + str(wavelens[index]),
                                        'units': 'sr-1',
                                        'wavelength': wavelens[index]})

# plotting
plt.plot(num_bands, test_deep, label='deep ' + str(round(h_deep * -1, 1)) + 'm')
plt.plot(num_bands, test_shallow, label='shallow ' + str(round(h_shallow * -1, 1)) + 'm')
plt.legend()
plt.xticks(num_bands, wavelens)
plt.xlabel('wavelength, nm')
plt.ylabel('Rrsw, sr^-1')
#plt.ylim([0, 0.04])
plt.show()